In [1]:
import sys
sys.path.insert(1, '../..')
import os
import re
import pandas as pd
from collections import Counter
from medicine import config

In [2]:
file_path = os.path.join(config.Path['data_dir'], '本草纲目(簡體)_Ver2.csv')
data = pd.read_csv(file_path, index_col=[0], engine='python', encoding='utf-8-sig')
data.columns

Index(['目錄', '篇名', '釋名', '內容', '部位', '部位內容', '主治', '氣味', '附方'], dtype='object')

In [3]:
propertys = data.loc[:,'氣味']
propertys[1:5]

1     甘，平，无毒。 \n
2     甘，平，无毒。 \n
3    甘，大寒，无毒。 \n
4     甘，平，无毒。 \n
Name: 氣味, dtype: object

In [4]:
pattern = '.*?[。；]|.*'

cuts = []
for p in propertys:
    if pd.isnull(p):
        continue
    match = re.match(pattern, p)
    if match:
        content = match.group()
        cuts_result = re.split('[，、；。]', content)
        cuts.extend(cuts_result)
print(set(cuts))

{'', '缺', '微甘', '小寒', '大温', '寒', '不可近目（《别录》）', '同根', '五种石脂', '淡', '（缺） ', '同谷', '同菜', '地浆解之 ', '令人吐泻', '小毒', '甘 ', '同叶', '有小毒（苏恭）', '微寒', '同茎', '缺 ', '同前', '原缺', '无 ', '同华', '同心', '香', '无毒（《本经》）', '有毒', '大寒', '甜', '同枳 ', '根同', '熟热', '与子同', '同金', '辛美微臭', '茎叶∶苦', '冷利', ' ', '性凉', '并 ', '小温', '时珍曰∶按∶《南越志》云∶无毒', '微咸', '微热', '至冷', '甘', '小冷', ' 无毒', '有大毒', '咸', '温', '滑利', '并甘', '四距者', '酸', '暖', '微酸', '滑', '凉', '同肉', '同苗', '同香', '微苦', '食之令人笑不止', '鸡死足不伸者', '缺平', '燔之', '腥', '同实', '无毒（保升）', '无毒', '辛', '大有毒', '同', '花∶甘', '微毒', '无毒 ', '食忌 诜曰∶鸡有五色者', '同皮', '微温', '涩', '有 ', '同上', '苦', '同 ', '与服石人相反（《开宝》）', '诸纸∶甘', '藏器曰∶寒', '玄鸡白首者', '平', '小苦', '同瓤', '吐人', '并酸', '微辛', '辛美', '同葱', '有微毒', '生凉', '热', '大热', '味如水牛肉', '同子', '冷', '臭', '六指者', '有小毒'}


In [5]:
from collections import defaultdict

In [6]:
pattern = '.*?[。；]|.*'
property_keys = ['cold','cool', 'warm', 'hot', 'neutral',
                 'sour', 'sweet', 'bitter', 'salty', 'pungent', 'poison']


for p in propertys:
    if pd.isnull(p):
        continue
    property_dict = dict.fromkeys(property_keys, 0)
    match = re.match(pattern, p)
    if match:
        content = re.sub('\s', '', match.group())
    if re.search('热', content):
        text = re.search('(大热)|(熟热)|(热)|(微热)', content).group()
        if text == '大热':
            property_dict['hot'] = 3
        elif  text == '熟热' or text == '热':
            property_dict['hot'] = 2
        elif  text == '微热':
            property_dict['hot'] = 1
            
    if re.search('温', content):
        text = re.search('(大温)|(温)|(小温)|(微温)', content).group()
        if text == '大温':
            property_dict['warm'] = 3
        elif  text == '温':
            property_dict['warm'] = 2
        elif  text == '小温' or text == '微温':
            property_dict['warm'] = 1
            
    if re.search('冷|凉', content):
        text = re.search('(冷利)|(至冷)|(冷)|(小冷)|(凉)', content).group()
        if text == '冷利' or text == '至冷':
            property_dict['cool'] = 3
        elif  text == '冷' or text == '凉':
            property_dict['cool'] = 2
        elif  text == '小冷':
            property_dict['cool'] = 1
            
    if re.search('寒', content):
        text = re.search('(大寒)|(寒)|(小寒)|(微寒)', content).group()
        if text == '大寒':
            property_dict['cold'] = 3
        elif  text == '寒':
            property_dict['cold'] = 2
        elif  text == '小寒' or text == '微寒':
            property_dict['cold'] = 1
        
    if re.search('平', content):
        property_dict['neutral'] = 2
        
    if re.search('酸', content):
        text = re.search('(并酸)|(酸)|(微酸)', content).group()
        if text == '并酸':
            property_dict['sour'] = 3
        elif  text == '酸':
            property_dict['sour'] = 2
        elif  text == '微酸':
            property_dict['sour'] = 1
            
    if re.search('甘', content):
        text = re.search('(并甘)|(甘)|(微甘)', content).group()
        if text == '并甘':
            property_dict['sweet'] = 3
        elif  text == '甘':
            property_dict['sweet'] = 3
        elif  text == '微甘':
            property_dict['sweet'] = 3
            
    if re.search('苦', content):
        text = re.search('(苦)|(微苦)|(小苦)', content).group()
        if text == '苦':
            property_dict['bitter'] = 2
        elif  text == '微苦' or text == '小苦':
            property_dict['bitter'] = 1
            
    if re.search('咸', content):
        text = re.search('(咸)|(微咸)', content).group()
        if text == '咸':
            property_dict['salty'] = 2
        elif  text == '微咸':
            property_dict['salty'] = 1
            
    if re.search('辛', content):
        text = re.search('(辛美)|(辛)|(微辛)', content).group()
        if text == '辛美' or text == '辛':
            property_dict['pungent'] = 2
        elif  text == '微辛':
            property_dict['pungent'] = 1
            
    if re.search('毒', content):
        text = re.search('(大毒)|(大有毒)|(有毒)|(微毒)|(小毒)|(无毒)', content).group()
        if text == '大毒' or text == '大有毒':
            property_dict['poison'] = 3
        elif  text == '有毒':
            property_dict['poison'] = 2
        elif  text == '微毒' or text == '小毒':
            property_dict['poison'] = 1
#     print(content, '\n', property_dict)



In [7]:
from collections import OrderedDict

In [8]:
pattern = '.*?[。；]|.*'
property_keys = ['cold','cool', 'warm', 'hot',
                 'neutral', 'sour', 'sweet', 'bitter',
                 'salty', 'pungent', 'poison']

medicine_dict = OrderedDict()
medicine_temp = None
for index, row in data.iterrows():
    property_dict = dict.fromkeys(property_keys, 0)
    medicine = re.sub('\s', '', row['篇名'])
    position = re.sub('\s', '', row['部位']) if not pd.isnull(row['部位']) else ''
    content = re.sub('\s', '', row['氣味']) if not pd.isnull(row['氣味']) else ''
    
    if medicine_temp != medicine:
        medicine_temp = medicine
        medicine_dict = OrderedDict()
    
    match = re.match(pattern, content)
    if match:
        content = match.group()
        
    if re.search('同', content):
        something = ''
        if re.search('(?:同([^。]))|(?:(.)同)', content):
            for search in re.search('(?:同([^。]))|(?:(.)同)', content).groups():
                if search is not None:
                    something = search
                    break
        
        if something == '' and len(medicine_dict) > 0:
            first_key = list(medicine_dict.items())[0][0]
            medicine_dict[position] = medicine_dict[first_key]
            
        elif len(medicine_dict) > 0:
            match_success = False
            for item in medicine_dict.keys():
                if re.search(something, item):
                    match_success = True
                    property_dict = medicine_dict[item]
                    break
            if match_success == False:
                property_dict = list(medicine_dict.items())[len(medicine_dict)-1][1]
            
    
    if re.search('热', content):
        text = re.search('(大热)|(熟热)|(热)|(微热)', content).group()
        if text == '大热':
            property_dict['hot'] = 3
        elif  text == '熟热' or text == '热':
            property_dict['hot'] = 2
        elif  text == '微热':
            property_dict['hot'] = 1
            
    if re.search('温', content):
        text = re.search('(大温)|(温)|(小温)|(微温)', content).group()
        if text == '大温':
            property_dict['warm'] = 3
        elif  text == '温':
            property_dict['warm'] = 2
        elif  text == '小温' or text == '微温':
            property_dict['warm'] = 1
            
    if re.search('冷|凉', content):
        text = re.search('(冷利)|(至冷)|(冷)|(小冷)|(凉)', content).group()
        if text == '冷利' or text == '至冷':
            property_dict['cool'] = 3
        elif  text == '冷' or text == '凉':
            property_dict['cool'] = 2
        elif  text == '小冷':
            property_dict['cool'] = 1
            
    if re.search('寒', content):
        text = re.search('(大寒)|(寒)|(小寒)|(微寒)', content).group()
        if text == '大寒':
            property_dict['cold'] = 3
        elif  text == '寒':
            property_dict['cold'] = 2
        elif  text == '小寒' or text == '微寒':
            property_dict['cold'] = 1
        
    if re.search('平', content):
        property_dict['neutral'] = 2
        
    if re.search('酸', content):
        text = re.search('(并酸)|(酸)|(微酸)', content).group()
        if text == '并酸':
            property_dict['sour'] = 3
        elif  text == '酸':
            property_dict['sour'] = 2
        elif  text == '微酸':
            property_dict['sour'] = 1
            
    if re.search('甘', content):
        text = re.search('(并甘)|(甘)|(微甘)', content).group()
        if text == '并甘':
            property_dict['sweet'] = 3
        elif  text == '甘':
            property_dict['sweet'] = 3
        elif  text == '微甘':
            property_dict['sweet'] = 3
            
    if re.search('苦', content):
        text = re.search('(苦)|(微苦)|(小苦)', content).group()
        if text == '苦':
            property_dict['bitter'] = 2
        elif  text == '微苦' or text == '小苦':
            property_dict['bitter'] = 1
            
    if re.search('咸', content):
        text = re.search('(咸)|(微咸)', content).group()
        if text == '咸':
            property_dict['salty'] = 2
        elif  text == '微咸':
            property_dict['salty'] = 1
            
    if re.search('辛', content):
        text = re.search('(辛美)|(辛)|(微辛)', content).group()
        if text == '辛美' or text == '辛':
            property_dict['pungent'] = 2
        elif  text == '微辛':
            property_dict['pungent'] = 1
            
    if re.search('毒', content):
        text = re.search('(大毒)|(大有毒)|(有毒)|(微毒)|(小毒)|(无毒)', content).group()
        if text == '大毒' or text == '大有毒':
            property_dict['poison'] = 3
        elif  text == '有毒':
            property_dict['poison'] = 2
        elif  text == '微毒' or text == '小毒':
            property_dict['poison'] = 1
    
    medicine_dict[position] = property_dict
    print(property_dict)

    

{'cold': 0, 'cool': 0, 'warm': 0, 'hot': 0, 'neutral': 2, 'sour': 0, 'sweet': 0, 'bitter': 0, 'salty': 2, 'pungent': 0, 'poison': 0}
{'cold': 0, 'cool': 0, 'warm': 0, 'hot': 0, 'neutral': 2, 'sour': 0, 'sweet': 3, 'bitter': 0, 'salty': 0, 'pungent': 0, 'poison': 0}
{'cold': 0, 'cool': 0, 'warm': 0, 'hot': 0, 'neutral': 2, 'sour': 0, 'sweet': 3, 'bitter': 0, 'salty': 0, 'pungent': 0, 'poison': 0}
{'cold': 3, 'cool': 0, 'warm': 0, 'hot': 0, 'neutral': 0, 'sour': 0, 'sweet': 3, 'bitter': 0, 'salty': 0, 'pungent': 0, 'poison': 0}
{'cold': 0, 'cool': 0, 'warm': 0, 'hot': 0, 'neutral': 2, 'sour': 0, 'sweet': 3, 'bitter': 0, 'salty': 0, 'pungent': 0, 'poison': 0}
{'cold': 2, 'cool': 0, 'warm': 0, 'hot': 0, 'neutral': 0, 'sour': 0, 'sweet': 3, 'bitter': 0, 'salty': 0, 'pungent': 0, 'poison': 0}
{'cold': 2, 'cool': 0, 'warm': 0, 'hot': 0, 'neutral': 0, 'sour': 0, 'sweet': 3, 'bitter': 0, 'salty': 0, 'pungent': 0, 'poison': 0}
{'cold': 0, 'cool': 2, 'warm': 0, 'hot': 0, 'neutral': 0, 'sour': 0, 

{'cold': 0, 'cool': 0, 'warm': 2, 'hot': 0, 'neutral': 0, 'sour': 0, 'sweet': 0, 'bitter': 0, 'salty': 0, 'pungent': 2, 'poison': 0}
{'cold': 0, 'cool': 0, 'warm': 0, 'hot': 0, 'neutral': 2, 'sour': 0, 'sweet': 3, 'bitter': 0, 'salty': 0, 'pungent': 0, 'poison': 0}
{'cold': 0, 'cool': 2, 'warm': 0, 'hot': 0, 'neutral': 0, 'sour': 0, 'sweet': 3, 'bitter': 0, 'salty': 0, 'pungent': 0, 'poison': 0}
{'cold': 0, 'cool': 2, 'warm': 0, 'hot': 0, 'neutral': 0, 'sour': 0, 'sweet': 3, 'bitter': 0, 'salty': 0, 'pungent': 0, 'poison': 0}
{'cold': 2, 'cool': 0, 'warm': 0, 'hot': 0, 'neutral': 0, 'sour': 0, 'sweet': 3, 'bitter': 0, 'salty': 0, 'pungent': 0, 'poison': 0}
{'cold': 2, 'cool': 0, 'warm': 0, 'hot': 0, 'neutral': 0, 'sour': 0, 'sweet': 3, 'bitter': 0, 'salty': 0, 'pungent': 0, 'poison': 0}
{'cold': 3, 'cool': 0, 'warm': 0, 'hot': 0, 'neutral': 0, 'sour': 0, 'sweet': 0, 'bitter': 2, 'salty': 0, 'pungent': 0, 'poison': 0}
{'cold': 2, 'cool': 0, 'warm': 0, 'hot': 0, 'neutral': 0, 'sour': 0, 

{'cold': 0, 'cool': 0, 'warm': 1, 'hot': 0, 'neutral': 0, 'sour': 2, 'sweet': 0, 'bitter': 0, 'salty': 0, 'pungent': 0, 'poison': 2}
{'cold': 0, 'cool': 0, 'warm': 0, 'hot': 0, 'neutral': 0, 'sour': 0, 'sweet': 3, 'bitter': 0, 'salty': 0, 'pungent': 0, 'poison': 1}
{'cold': 0, 'cool': 0, 'warm': 0, 'hot': 0, 'neutral': 0, 'sour': 0, 'sweet': 3, 'bitter': 0, 'salty': 0, 'pungent': 0, 'poison': 1}
{'cold': 0, 'cool': 0, 'warm': 0, 'hot': 0, 'neutral': 0, 'sour': 0, 'sweet': 3, 'bitter': 0, 'salty': 0, 'pungent': 0, 'poison': 1}
{'cold': 1, 'cool': 0, 'warm': 0, 'hot': 0, 'neutral': 0, 'sour': 0, 'sweet': 0, 'bitter': 0, 'salty': 2, 'pungent': 0, 'poison': 2}
{'cold': 0, 'cool': 0, 'warm': 2, 'hot': 0, 'neutral': 0, 'sour': 0, 'sweet': 3, 'bitter': 0, 'salty': 0, 'pungent': 0, 'poison': 2}
{'cold': 0, 'cool': 0, 'warm': 0, 'hot': 0, 'neutral': 0, 'sour': 0, 'sweet': 0, 'bitter': 0, 'salty': 0, 'pungent': 0, 'poison': 0}
{'cold': 0, 'cool': 0, 'warm': 0, 'hot': 0, 'neutral': 0, 'sour': 0, 